# pba-modul4-id-text-similarity-clustering-recommender

In [1]:
import os
import re
import sys
import string
import modSpellChecker_1 as sc
from contractions_1 import CONTRACTION_MAP
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import pandas as pd
import operator

Deklarasi Fungsi untuk Normalisasi Teks

In [2]:
character = ['z','y','x','w','v','u','t','s','r','q','p','o','n','m','l','k','j','i','h','g','f','e','d',',',';',':','-','...','?','!', '(',')','[',']','{','}','<','>', '"','/','\'','#','-','@']

def repeatcharNormalize(text):
    for i in range(len(character)):
        charac_long = 5
        while charac_long >= 2:
            char = character[i] * charac_long
            text = text.replace(char, character[i])
            charac_long -= 1
    return text

def spellNormalize(text):
    spellCheck = []
    for i in text:
        if i not in character:
            j = sc.correction(i)
            spellCheck.append(j)
        else:
            spellCheck.append(i)
    return spellCheck

def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    return tokens

def expand_contractions(text, contraction_mapping):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match) if contraction_mapping.get(match) else contraction_mapping.get(match.lower())
        expanded_contraction = first_char + expanded_contraction[1:]
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def stemmer_text(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = stemmer.stem(text)
    return text

In [3]:
def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('',token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [4]:
factory = StopWordRemoverFactory()
stopword_list = factory.get_stop_words()

In [5]:
def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [6]:
def normalize_corpus(corpus, tokenize=False):
    normalized_corpus = []
    for text in corpus:
        text = expand_contractions(text, CONTRACTION_MAP)
        text = stemmer_text(text)
        text = remove_special_characters(text)
        text = repeatcharNormalize(text)
        text = remove_stopwords(text)
        normalized_corpus.append(text)
        if tokenize:
            text = tokenize_text(text)
            text = spellNormalize(text)
            normalized_corpus.append(text)
    return normalized_corpus

Penyiapan Data Teks

In [7]:
dataset = pd.read_csv('detik_2023-10-18_08-30-26.csv')
dataset

,title,author,category,date,content,link
0,Penonton Jagokan Marc Marquez di Sprint Race M...,Okdwitya Karina Sari,MotoGP,"Sabtu, 14 Okt 2023 11:10 WIB","Meski sedang tak oke-oke amat, Marc Marquez ma...",https://sport.detik.com/moto-gp/d-6981885/peno...
1,Cara Anthony Ginting Balikin Motivasi Usai Gag...,Mercy Raya,Raket,"Rabu, 11 Okt 2023 21:45 WIB",Anthony Sinisuka Ginting mengaku sulit move on...,https://sport.detik.com/raket/d-6977523/cara-a...
2,Hasil Kualifikasi MotoGP Indonesia 2023: Luca ...,Rifqi Ardita Widianto,MotoGP,"Sabtu, 14 Okt 2023 10:28 WIB",Luca Marini secara impresif merebut pole posit...,https://sport.detik.com/moto-gp/d-6981874/hasi...
3,Kisah Fans Diundang Khusus Race Director untuk...,Okdwitya Karina Sari,MotoGP,"Sabtu, 14 Okt 2023 12:10 WIB","Bahagia betul Ira (42). Dia dan putranya, Mika...",https://sport.detik.com/moto-gp/d-6981921/kisa...
4,Ini Seragam Baru Kontingen Indonesia di Asian ...,Mercy Raya,Sport Lain,"Sabtu, 14 Okt 2023 11:30 WIB",Kontingen Indonesia yang akan tampil di Asian ...,https://sport.detik.com/sport-lain/d-6981910/i...
...,...,...,...,...,...,...
995,"Dapat Poin Usai Crash, Alex Marquez: MotoGP Ca...",Lucas Aditya,MotoGP,"Senin, 04 Sep 2023 17:30 WIB","Rider Gresini, Alex Marquez, menyebut balapan ...",https://sport.detik.com/moto-gp/d-6912778/dapa...
996,"Jadwal Lengkap China Open 2023, Tunggal Putra ...",Mercy Raya,Raket,"Senin, 04 Sep 2023 14:50 WIB",Turnamen bulutangkis China Open 2023 akan dise...,https://sport.detik.com/raket/d-6912459/jadwal...
997,Rekor! Penonton FIBA World Cup Jakarta Tembus ...,Mercy Raya,Basket,"Senin, 04 Sep 2023 15:55 WIB",Penonton FIBA World Cup Jakarta menembus angka...,https://sport.detik.com/basket/d-6912672/rekor...
998,Kata Dokter Soal Cedera Francesco Bagnaia,"detikTV, dtv",Detiktv,"Senin, 04 Sep 2023 08:43 WIB",Direktur Medis MotoGP dr. Angel Charte menyebu...,https://sport.detik.com/detiktv/d-6911700/kata...


In [8]:
feature = dataset.iloc[:,0]

In [9]:
feature[0:10]

0    Penonton Jagokan Marc Marquez di Sprint Race M...
1    Cara Anthony Ginting Balikin Motivasi Usai Gag...
2    Hasil Kualifikasi MotoGP Indonesia 2023: Luca ...
3    Kisah Fans Diundang Khusus Race Director untuk...
4    Ini Seragam Baru Kontingen Indonesia di Asian ...
5    Hasil Sprint Race MotoGP Mandalika 2023: Jorge...
6    Rok Super Final 2023: Qarrar Posisi Tiga Besar...
7    Sumatra Selatan Raih Dua Medali di Hari Pertam...
8    Haornas 2023: Ketua PBSI Jabar Raih Penghargaa...
9    Fakta Seputar Futsal: Sejarah hingga Bedanya d...
Name: title, dtype: object

In [10]:
norm_corpus = normalize_corpus(feature)
len(norm_corpus)

1000

Ekstraksi Fitur dengan TFIDF

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(1000, 1593)

1.4 Penghitungan Text Similarity

1.4.1 Metode Cosine Similarity

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.009246,0.030702,0.226892,0.011371,0.518466,0.007699,0.007541,0.009625,0.0,...,0.000000,0.0,0.358713,0.205988,0.010930,0.073015,0.008182,0.114394,0.0,0.009110
1,0.009246,1.000000,0.007916,0.000000,0.118898,0.008516,0.006938,0.006795,0.008673,0.0,...,0.120595,0.0,0.007385,0.000000,0.009849,0.062164,0.007372,0.000000,0.0,0.008209
2,0.030702,0.007916,1.000000,0.023834,0.038135,0.064438,0.065006,0.006456,0.008241,0.0,...,0.000000,0.0,0.055880,0.000000,0.009357,0.020759,0.007005,0.000000,0.0,0.007800
3,0.226892,0.000000,0.023834,1.000000,0.000000,0.208968,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.094433,0.000000,0.000000,0.026368,0.000000,0.000000,0.0,0.000000
4,0.011371,0.118898,0.038135,0.000000,1.000000,0.010473,0.008532,0.008357,0.010667,0.0,...,0.000000,0.0,0.009082,0.000000,0.012112,0.000000,0.009067,0.000000,0.0,0.010096


In [13]:
article_list = dataset['title'].values
article_list, article_list.shape

(array(['Penonton Jagokan Marc Marquez di Sprint Race MotoGP Mandalika 2023',
        'Cara Anthony Ginting Balikin Motivasi Usai Gagal di Asian Games 2023',
        'Hasil Kualifikasi MotoGP Indonesia 2023: Luca Marini Rebut Pole!',
        'Kisah Fans Diundang Khusus Race Director untuk Nonton MotoGP Mandalika',
        'Ini Seragam Baru Kontingen Indonesia di Asian Para Games 2023',
        'Hasil Sprint Race MotoGP Mandalika 2023: Jorge Martin Juara!',
        'Rok Super Final 2023: Qarrar Posisi Tiga Besar di Kualifikasi',
        'Sumatra Selatan Raih Dua Medali di Hari Pertama Kejurnas Berkuda 2023',
        'Haornas 2023: Ketua PBSI Jabar Raih Penghargaan Pembina Olahraga',
        'Fakta Seputar Futsal: Sejarah hingga Bedanya dengan Sepakbola',
        'Berlian/Malika Juara Ganda Anak-anak Putri BNI Sirnas Pangkalpinang',
        'Denmark Open 2023: The Daddies Menang Dramatis atas Pram/Yere',
        'MotoGP Indonesia 2023: Sudah Seru Sejak di Bandara',
        'Kamaru Usman:

In [15]:
article_idx = np.where(article_list=='Penonton Jagokan Marc Marquez di Sprint Race MotoGP Mandalika 2023')[0][0]
article_idx

0

Pencarian Dokumen Similarity

In [16]:
article_similarities = doc_sim_df.iloc[article_idx].values
article_similarities

array([1.        , 0.00924621, 0.0307019 , 0.22689249, 0.01137136,
       0.51846601, 0.00769925, 0.00754068, 0.00962516, 0.        ,
       0.        , 0.00767235, 0.04655051, 0.16912993, 0.02622197,
       0.        , 0.03784909, 0.20918365, 0.        , 0.01157465,
       0.        , 0.        , 0.01005995, 0.70486677, 0.00829595,
       0.06370241, 0.00815292, 0.        , 0.18096292, 0.00740516,
       0.03456657, 0.00868731, 0.00957676, 0.00917775, 0.        ,
       0.00836161, 0.        , 0.1892964 , 0.        , 0.25404946,
       0.00888641, 0.        , 0.00998432, 0.        , 0.03424003,
       0.32260556, 0.03297466, 0.03597609, 0.23048729, 0.        ,
       0.00871485, 0.01143717, 0.00895274, 0.33455077, 0.21200413,
       0.11240685, 0.        , 0.02305684, 0.        , 0.05968624,
       0.02463762, 0.        , 0.        , 0.        , 0.26234596,
       0.        , 0.        , 0.00783882, 0.03053658, 0.0075132 ,
       0.00720528, 0.        , 0.02977861, 0.        , 0.     

In [17]:
similar_article_idxs = np.argsort(-article_similarities)[1:6]
similar_article_idxs

array([ 23, 860, 239,   5, 989])

In [18]:
similar_article = article_list[similar_article_idxs]
similar_article

array(['Sprint Race MotoGP Mandalika 2023: Marc Marquez Langsung Crash!',
       "Marc Marquez 'Main Aman' di Sprint Race MotoGP Catalunya",
       'Marc Marquez Belum Berjodoh dengan MotoGP Mandalika',
       'Hasil Sprint Race MotoGP Mandalika 2023: Jorge Martin Juara!',
       'Hujan Deras Tunda Sprint Race MotoGP India 2023'], dtype=object)

Deklarasi Fungsi untuk Pencarian Dokumen Similarity

In [53]:
def article_recommender(article_title, articles=article_list,doc_sims=doc_sim_df):
    article_idx = np.where(articles == article_title)[0][0]
    print(article_idx)
    article_similarities = doc_sims.iloc[article_idx].values
    similar_article_idxs = np.argsort(-article_similarities)[1:6]
    similar_articles = articles[similar_article_idxs]
    return similar_articles

In [59]:
popular_articles = ['Penonton Jagokan Marc Marquez di Sprint Race MotoGP Mandalika 2023',
'MotoGP San Marino 2023: Dani Pedrosa Finis P4, Teringat Rossi',
'Sprint Race MotoGP Mandalika 2023: Marc Marquez Langsung Crash!'
]

In [60]:
for article in popular_articles:
    print('Article:', article)
    print('Top 5 recommended Article:',article_recommender(article_title=article))
    print()

Article: Penonton Jagokan Marc Marquez di Sprint Race MotoGP Mandalika 2023
0
Top 5 recommended Article: ['Sprint Race MotoGP Mandalika 2023: Marc Marquez Langsung Crash!'
 "Marc Marquez 'Main Aman' di Sprint Race MotoGP Catalunya"
 'Marc Marquez Belum Berjodoh dengan MotoGP Mandalika'
 'Hasil Sprint Race MotoGP Mandalika 2023: Jorge Martin Juara!'
 'Hujan Deras Tunda Sprint Race MotoGP India 2023']

Article: MotoGP San Marino 2023: Dani Pedrosa Finis P4, Teringat Rossi
16
Top 5 recommended Article: ['Jadwal MotoGP San Marino 2023 Hari Ini: Berpacu di Misano'
 'MotoGP San Marino 2023: Martin Juara Ungguli Dua Murid Rossi'
 'Martin Juara MotoGP San Marino 2023, Dua Murid Rossi Sengit'
 'Akhir Pekan Sempurna Jorge Martin di MotoGP San Marino 2023'
 "Sprint Race MotoGP San Marino: Martin Juara, Pedrosa 'Ganggu' Bagnaia"]

Article: Sprint Race MotoGP Mandalika 2023: Marc Marquez Langsung Crash!
23
Top 5 recommended Article: ['Penonton Jagokan Marc Marquez di Sprint Race MotoGP Mandalika 20

Metode BM25 Similarity

In [40]:
"""
Data:
-----
.. data:: PARAM_K1 - Free smoothing parameter for BM25.
.. data:: PARAM_B - Free smoothing parameter for BM25.
.. data:: EPSILON - Constant used for negative idf of document in corpus.
"""

import math
from six import iteritems
from six.moves import xrange

PARAM_K1 = 2.5
PARAM_B = 0.85
EPSILON = 0.2

class BM25(object):
    """Implementation of Best Matching 25 ranking function.
    Attributes
    ----------
    corpus_size : int
        Size of corpus (number of documents).
    avgdl : float
        Average length of document in `corpus`.
    corpus : list of list of str
        Corpus of documents.
    f : list of dicts of int
        Dictionary with terms frequencies for each document in `corpus`.
    df : dict
        Dictionary with terms frequencies for whole `corpus`.
    idf : dict
        Dictionary with inversed terms frequencies for while `corpus`.
    doc_len : list of int
        List of document lengths.
    """

    def __init__(self, corpus):
        """
        Parameters
        ----------
        corpus : list of list of str
            Given corpus.
        """
        self.corpus_size = len(corpus)
        self.avgdl = sum(float(len(x)) for x in corpus) / self.corpus_size
        self.corpus = corpus
        self.f = []
        self.df = {}
        self.idf = {}
        self.doc_len = []
        self.initialize()

    def initialize(self):
        for document in self.corpus:
            frequencies = {}
            self.doc_len.append(len(document))
            for word in document:
                if word not in frequencies:
                    frequencies[word] = 0
                frequencies[word] += 1
            self.f.append(frequencies)

            for word, freq in iteritems(frequencies):
                if word not in self.df:
                    self.df[word] = 0
                self.df[word] += 1

        for word, freq in iteritems(self.df):
            self.idf[word] = math.log(self.corpus_size - freq + 0.5) - math.log(freq)

    def get_score(self, document, index, average_idf):
        """Computes BM25 score of given `document` in relation to item of corpus
        Parameters
        ----------
        document : list of str
            Document to be scored.
        index : int
            Index of document in corpus selected to score with `document`.
        average_idf : float
            Average idf in corpus.
        Returns
        -------
        float
            BM25 score.
        """
        score = 0
        for word in document:
            if word not in self.f[index]:
                continue
            idf = self.idf[word] if self.idf[word] >= 0 else EPSILON * average_idf
            score += (idf * self.f[index][word] * (PARAM_K1 + 1)) / (self.f[index][word] + 
                                                                     PARAM_K1 * (1 - PARAM_B + PARAM_B *self.doc_len[index] / self.avgdl))
        return score

    def get_scores(self, document, average_idf):
        scores = []
        for index in xrange(self.corpus_size):
            score = self.get_score(document, index, average_idf)
            scores.append(score)
        return scores


def get_bm25_weights(corpus):
    """Returns BM25 scores (weights) of documents in corpus.
    Each document has to be weighted with every document in given corpus.
    Parameters
    ----------
    corpus : list of list of str
        Corpus of documents.
    Returns
    -------
    list of list of float
        BM25 scores.
    Examples
    --------
    >>> from gensim.summarization.bm25 import get_bm25_weights
    >>> corpus = [
    ...     ["black", "cat", "white", "cat"],
    ...     ["cat", "outer", "space"],
    ...     ["wag", "dog"]
    ... ]
    >>> result = get_bm25_weights(corpus)
    """
    bm25 = BM25(corpus)
    average_idf = sum(float(val) for val in bm25.idf.values()) / len(bm25.idf)

    weights = []
    for doc in corpus:
        scores = bm25.get_scores(doc, average_idf)
        weights.append(scores)

    return weights

In [41]:
a = [nltk.word_tokenize(doc) for doc in norm_corpus]

In [42]:
norm_corpus_tokens = np.asarray(a, dtype="object")

In [43]:
norm_corpus_tokens[:3]

array([list(['tonton', 'jago', 'marc', 'marquez', 'sprint', 'race', 'motogp', 'mandalika', '2023']),
       list(['cara', 'anthony', 'ginting', 'balikin', 'motivasi', 'usai', 'gagal', 'asi', 'games', '2023']),
       list(['hasil', 'kualifikasi', 'motogp', 'indonesia', '2023', 'luca', 'marini', 'rebut', 'pole'])],
      dtype=object)

Penghitungan BM25 Weights untuk Document

In [44]:
%%time
wts = get_bm25_weights(norm_corpus_tokens)

CPU times: user 902 ms, sys: 16 ms, total: 918 ms
Wall time: 918 ms


In [45]:
bm25_wts_df = pd.DataFrame(wts)
bm25_wts_df.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,26.419987,1.088599,2.443785,6.941504,1.354138,11.777800,1.164732,1.164732,1.164732,0.0,...,0.000000,0.0,10.550897,5.605346,1.252314,4.014910,1.164732,4.435306,0.0,1.164732
1,1.164732,33.683410,1.164732,0.000000,4.368830,1.164732,1.164732,1.164732,1.164732,0.0,...,3.995824,0.0,1.021809,0.000000,1.252314,3.394892,1.164732,0.000000,0.0,1.164732
2,2.443785,1.088599,28.226468,1.279053,3.125100,4.685795,4.836296,1.164732,1.164732,0.0,...,0.000000,0.0,4.110807,0.000000,1.252314,1.375232,1.164732,0.000000,0.0,1.164732
3,6.941504,0.000000,1.279053,42.162855,0.000000,6.941504,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,4.154245,0.000000,0.000000,1.375232,0.000000,0.000000,0.0,0.000000
4,1.164732,3.512127,2.687986,0.000000,23.136804,1.164732,1.164732,1.164732,1.164732,0.0,...,0.000000,0.0,1.021809,0.000000,1.252314,0.000000,1.164732,0.000000,0.0,1.164732


In [46]:
for article in popular_articles:
    print('Article:', article)
    print('Top 5 recommended Articles:',article_recommender(article_title=article, doc_sims=bm25_wts_df))
    print()

Article: Andil Penting Sirkuit Mandalika Bawa Pembalap Indonesia Sukses di Asia
Top 5 recommended Articles: []

Article: 5 Fakta Memalukan MU yang Terjungkal di Liga Champions
Top 5 recommended Articles: []

Article: Hasil imbang yang pahit bagi AC Milan
Top 5 recommended Articles: []

Article: 5 Fakta Jonatan vs Kunlavut di BWF World Tour Finals
Top 5 recommended Articles: []

Article: MU Sudah Jatuh Tertimpa Tangga, Dibantai Realita, Ditunggu Liverpool
Top 5 recommended Articles: []

